In [117]:
import numpy as np
import json

In [118]:
#import distance data
with open(r'json/distance.json') as f:
    city_distances = json.load(f)

#import input data
with open(r'json/input.json') as f:
    city_data = json.load(f)

In [119]:
#get distance function
def get_distance(start:str, end:str, data:dict):
    dis = data[start][end] 
    if (type(data[start][end]) == str):
        return 0
    else:
        return dis
    

#get closest depot
def get_depot(city:str, data:dict):
    depots = ['Goderich', 'Toronto', 'Picton']
    lowest_dist = 1E9
    closest_depot = ""

    for depot in depots:
        dist = get_distance(city, depot, data)
        
        if (dist != 0 and dist < lowest_dist):
            lowest_dist = dist
            closest_depot = depot
    return closest_depot

In [123]:
class Truck:
    def __init__(self, starting_depot) -> None:
        
        # Truck constants
        self.hourly_wage = 20
        self.overtime_wage = 30
        self.speed = 100
        self.cost_per_km = 0.68 + self.hourly_wage * self.speed / 100
        self.cost_per_km_overtime = 0.68 + self.overtime_wage * self.speed / 100
        
        # Truck variables
        self.home_depot = starting_depot
        self.current_city = self.home_depot
        self.rest = False
        self.distance_driven_day = 0
        self.distance_driven_week = 0
        self.day_cost = 0
        self.total_cost = 0
        self.packages = 5

        # Time variables
        self.current_day = "Monday"

        # Logging
        self.deliveries_info = {
            "Monday": [],
            "Tuesday": [],
            "Wednesday": [],
            "Thursday": [],
            "Friday": [],
        }

        # To output
        self.output_dictionary = {
            "Total_cost": self.total_cost,
            "Monday": {
                
            }
        }

    def reset_day(self):
        self.distance_driven_day = 0
        self.day_cost = 0

    def generate_day_info(self, day_distance:float, day_cost:float, day_deliveries:list) -> None:
        day_info = {
                "Total_distance": day_distance,
                "Total_cost": day_cost,
        }
        i = 1
        for delivery in day_deliveries:
            day_info[f'Delivery_{i}'] = delivery
            i += 1
        
    def generate_delivery_info(self, start_t, end_t, start_city, end_city, delivery_type, start_quant, end_quant, city_distances) -> dict:
        delivery_info = {
            "Start_time": start_t,
            "End_time": end_t,
            "Start_destination":start_city,
            "End_destination":end_city,
            "Distance": get_distance(start_city, end_city, city_distances),
            "Delivery_type":delivery_type,
            "Start_quantity":start_quant,
            "End_quantity": end_quant,
        }
        return delivery_info

    def generate_truck_output(self) -> dict:
        output_dict = {
            "Total_cost": self.total_cost
        }
        weekdays = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
        for weekday in weekdays:
            output_dict[weekday] = self.generate_day_info()

        return output_dict

    def move_truck(self, new_city:str, city_distances:dict, city_data:dict, deliver:bool=True):
        # Calculate cost
        print(city_data)
        dist = get_distance(self.current_city, new_city, city_distances)
        trip_cost = dist * self.cost_per_km

        # Update truck 
        old_city = self.current_city
        self.distance_driven_day += dist
        self.distance_driven_week += dist
        self.current_city = new_city
        
        # Deliver packages 
        start_packages = self.packages
        if deliver:
            num_packages_req = city_data[new_city]
            if self.packages <= num_packages_req:
                city_data[new_city] -= self.packages
                self.packages = 0
            else:
                self.packages -= city_data[new_city]
                city_data[new_city] = 0
        
        # Load up on packages 
        else:
            self.packages = 5
        
        # Create delivery log
        delivery_type = "Delivery"
        if not deliver:
            delivery_type = "Re-loading"
        
        self.deliveries_info[self.current_day] = (self.generate_delivery_info(0, 0, old_city, new_city, delivery_type, start_packages, self.packages, city_distances))

        #return (self.generate_delivery_info(0, 0, old_city, new_city, delivery_type, start_packages, self.packages, city_distances))
            

    def kill_trucker(self) -> None:
        self.rest = True


In [124]:
depots = ['Goderich', 'Toronto', 'Picton']
for city in city_distances.keys():
    lowest_dist = 1E9
    closest_depot = ""
    for depot in depots:
        dist = get_distance(city, depot, city_distances)
        
        if (dist != 0 and dist < lowest_dist):
            lowest_dist = dist
            closest_depot = depot

    print(f'{city} belongs to {closest_depot}')

Tilbury belongs to Goderich
Mississauga belongs to Toronto
Cornwall belongs to Picton
London belongs to Goderich
Windsor belongs to Goderich
Niagara_Falls belongs to Toronto
Barrie belongs to Toronto
Kingston belongs to Picton
Huntsville belongs to Toronto
North_Bay belongs to Toronto
Goderich belongs to Toronto
Toronto belongs to Picton
Picton belongs to Toronto


In [125]:
truck = Truck("Toronto")
truck.move_truck("Mississauga", city_distances, city_data, True)

{'Tilbury': 3, 'Mississauga': 4, 'Cornwall': 9, 'London': 13, 'Windsor': 5, 'Niagara_Falls': 13, 'Barrie': 18, 'Kingston': 18, 'Huntsville': 12, 'North_Bay': 5}


{'Start_time': 0,
 'End_time': 0,
 'Start_destination': 'Toronto',
 'End_destination': 'Mississauga',
 'Distance': 28,
 'Delivery_type': 'Delivery',
 'Start_quantity': 5,
 'End_quantity': 1}